In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import os

In [ ]:
print(os.getcwd())

/content


In [ ]:
%cd /content/gdrive/MyDrive/Masters/Year_1/Semester_2/CS_505/Project/NN

/content/gdrive/MyDrive/Masters/Year_1/Semester_2/CS_505/Project/NN


In [ ]:
from models.model_baselines import BaselineClassifier

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from typing import List, Dict, Optional, Tuple, Any
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler 

In [ ]:
print(tf.test.gpu_device_name())

/device:GPU:0


In [ ]:
%ls

'Copy of MT5_final_embeddings.ipynb'   mlp_tfidf_cv_2
'MD5_1 (1).ipynb'                      models/
 MD5_1_English.ipynb                   MT5_final_embeddings.ipynb
 mlp_tfidf_cv                          Untitled


In [ ]:
train_df = pd.read_csv('../Data/Train.csv')

In [ ]:
le = LabelEncoder()
train_df.Label = le.fit_transform(train_df.Label)

In [ ]:
Y_train = np_utils.to_categorical(train_df['Label'], np.unique(train_df['Label']).shape[0])

In [ ]:
def get_model(size):
  print('Building model...')
  model = Sequential()
  model.add(Dense(2048, input_shape=(size,)))
  model.add(Activation('relu'))
  model.add(Dropout(0.2))
  model.add(Dense(20))
  model.add(Activation('softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
  #make one, takes in, double that amount, english and chichewa

In [ ]:
def get_results(model,epochs,train_dataset,test_dataset,y_test):
  model.fit(train_dataset,
            epochs=epochs, batch_size=32,
            verbose=1)
  print(model.evaluate(test_dataset))
  ans = model.predict(test_dataset)
  print(accuracy_score(y_test, np_utils.to_categorical(np.argmax(ans,axis=1),num_classes=20)))
  print(classification_report(y_test, np_utils.to_categorical(np.argmax(ans,axis=1),num_classes=20)))

In [ ]:
print(Y_train.shape)
print(train_df.shape)

(1436, 20)
(1436, 3)


# Training with TFIDF Vectorizer

In [ ]:
kf = KFold(n_splits=3, random_state=42, shuffle=True)

In [ ]:
for train_index, test_index in kf.split(train_df.Text):
  X_train, X_test = train_df.iloc[train_index].Text, train_df.iloc[test_index].Text
  y_train, y_test = Y_train[train_index], Y_train[test_index]
  #print(train_index)
  #print(train_index.shape)
  #break
  tfidf = TfidfVectorizer()
  tfidf_train = tfidf.fit_transform(X_train).toarray()
  tfidf_test = tfidf.transform(X_test).toarray()
  train_dataset = tf.data.Dataset.from_tensor_slices((tfidf_train, y_train))

  test_dataset = tf.data.Dataset.from_tensor_slices((tfidf_test, y_test))

  BATCH_SIZE = 32
  SHUFFLE_BUFFER_SIZE = 100

  train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
  test_dataset = test_dataset.batch(BATCH_SIZE)
  get_results(get_model(tfidf_train.shape[1]),50,train_dataset,test_dataset,y_test)

Building model...
Epoch 1/50
30/30 [==============================] - 2s 19ms/step - loss: 2.5245 - accuracy: 0.2153
Epoch 2/50
30/30 [==============================] - 1s 23ms/step - loss: 1.1403 - accuracy: 0.7921
Epoch 3/50
30/30 [==============================] - 1s 25ms/step - loss: 0.2571 - accuracy: 0.9728
Epoch 4/50
30/30 [==============================] - 1s 22ms/step - loss: 0.0440 - accuracy: 0.9990
Epoch 5/50
30/30 [==============================] - 1s 26ms/step - loss: 0.0157 - accuracy: 1.0000
Epoch 6/50
30/30 [==============================] - 1s 20ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 7/50
30/30 [==============================] - 1s 26ms/step - loss: 0.0062 - accuracy: 1.0000
Epoch 8/50
30/30 [==============================] - 1s 21ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 9/50
30/30 [==============================] - 1s 21ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 10/50
30/30 [==============================] - 1s 19ms/step - loss: 0.0027 - accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Building model...
Epoch 1/50
30/30 [==============================] - 1s 18ms/step - loss: 2.5084 - accuracy: 0.2268
Epoch 2/50
30/30 [==============================] - 1s 18ms/step - loss: 1.0906 - accuracy: 0.8130
Epoch 3/50
30/30 [==============================] - 1s 19ms/step - loss: 0.2476 - accuracy: 0.9781
Epoch 4/50
30/30 [==============================] - 1s 19ms/step - loss: 0.0475 - accuracy: 0.9969
Epoch 5/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0196 - accuracy: 0.9969
Epoch 6/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0141 - accuracy: 0.9958
Epoch 7/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0108 - accuracy: 0.9969
Epoch 8/50
30/30 [==============================] - 1s 19ms/step - loss: 0.0098 - accuracy: 0.9969
Epoch 9/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0090 - accuracy: 0.9969
Epoch 10/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0069 - accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Building model...
Epoch 1/50
30/30 [==============================] - 2s 27ms/step - loss: 2.5114 - accuracy: 0.2338
Epoch 2/50
30/30 [==============================] - 1s 28ms/step - loss: 1.0835 - accuracy: 0.8027
Epoch 3/50
30/30 [==============================] - 1s 27ms/step - loss: 0.2521 - accuracy: 0.9760
Epoch 4/50
30/30 [==============================] - 1s 26ms/step - loss: 0.0453 - accuracy: 0.9990
Epoch 5/50
30/30 [==============================] - 1s 24ms/step - loss: 0.0196 - accuracy: 0.9990
Epoch 6/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0127 - accuracy: 0.9990
Epoch 7/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0096 - accuracy: 0.9990
Epoch 8/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0067 - accuracy: 0.9990
Epoch 9/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0065 - accuracy: 0.9990
Epoch 10/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0036 - accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#f1 macro 47 43 47
#accuracy 64 60 64

In [ ]:
print((47+43+47)/3)
print((64+60+64)/3)

45.666666666666664
62.666666666666664


# Training with Count Vectorizer

In [ ]:
for train_index, test_index in kf.split(train_df.Text):
  X_train, X_test = train_df.iloc[train_index].Text, train_df.iloc[test_index].Text
  y_train, y_test = Y_train[train_index], Y_train[test_index]
  
  cv = CountVectorizer()
  cv_train = cv.fit_transform(X_train).toarray()
  cv_test = cv.transform(X_test).toarray()
  train_dataset = tf.data.Dataset.from_tensor_slices((cv_train, y_train))

  test_dataset = tf.data.Dataset.from_tensor_slices((cv_test, y_test))

  BATCH_SIZE = 32
  SHUFFLE_BUFFER_SIZE = 100

  train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
  test_dataset = test_dataset.batch(BATCH_SIZE)
  get_results(get_model(cv_train.shape[1]),50,train_dataset,test_dataset,y_test)

Building model...
Epoch 1/50
30/30 [==============================] - 1s 14ms/step - loss: 1.8834 - accuracy: 0.4974
Epoch 2/50
30/30 [==============================] - 0s 15ms/step - loss: 0.0652 - accuracy: 0.9896
Epoch 3/50
30/30 [==============================] - 0s 14ms/step - loss: 0.0067 - accuracy: 1.0000
Epoch 4/50
30/30 [==============================] - 0s 15ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 5/50
30/30 [==============================] - 0s 14ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 6/50
30/30 [==============================] - 0s 15ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 7/50
30/30 [==============================] - 0s 14ms/step - loss: 8.1218e-04 - accuracy: 1.0000
Epoch 8/50
30/30 [==============================] - 0s 14ms/step - loss: 5.8166e-04 - accuracy: 1.0000
Epoch 9/50
30/30 [==============================] - 0s 15ms/step - loss: 4.4630e-04 - accuracy: 1.0000
Epoch 10/50
30/30 [==============================] - 0s 14ms/step - loss: 3.592

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Building model...
Epoch 1/50
30/30 [==============================] - 1s 18ms/step - loss: 1.9130 - accuracy: 0.4775
Epoch 2/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0955 - accuracy: 0.9885
Epoch 3/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0230 - accuracy: 0.9969
Epoch 4/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0152 - accuracy: 0.9958
Epoch 5/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0123 - accuracy: 0.9958
Epoch 6/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0166 - accuracy: 0.9969
Epoch 7/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0089 - accuracy: 0.9969
Epoch 8/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0124 - accuracy: 0.9969
Epoch 9/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0056 - accuracy: 0.9990
Epoch 10/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0119 - accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Building model...
Epoch 1/50
30/30 [==============================] - 1s 18ms/step - loss: 1.8899 - accuracy: 0.4875
Epoch 2/50
30/30 [==============================] - 1s 19ms/step - loss: 0.0864 - accuracy: 0.9896
Epoch 3/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0175 - accuracy: 0.9990
Epoch 4/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0184 - accuracy: 0.9990
Epoch 5/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0085 - accuracy: 0.9990
Epoch 6/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0098 - accuracy: 0.9990
Epoch 7/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0082 - accuracy: 0.9990
Epoch 8/50
30/30 [==============================] - 1s 19ms/step - loss: 0.0123 - accuracy: 0.9990
Epoch 9/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0033 - accuracy: 0.9990
Epoch 10/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0109 - accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#f1 macro 49 43 48
#accuracy 64 58 63

In [ ]:
print((49+43+48)/3)
print((64+58+63)/3)

46.666666666666664
61.666666666666664


# Training with MD5

In [ ]:
for train_index, test_index in kf.split(train_df.Text):
  print(train_index[:10])

[0 1 2 3 4 5 6 7 8 9]
[ 1  4  8 10 11 13 14 15 16 17]
[ 0  2  3  5  6  7  9 10 12 15]


In [ ]:
chichewa_embd = np.load('../Data/' + 'embeddings_chichewa_512_final.npy')

In [ ]:
english_embd = np.load('../Data/' + 'embeddings_english_512_final.npy')

In [ ]:
english_embd.shape

(1436, 512)

In [ ]:
final_comb_arr = np.hstack((chichewa_embd,english_embd))

In [ ]:
final_comb_arr.shape

(1436, 1024)

In [ ]:
for train_index, test_index in kf.split(final_comb_arr):
  print(train_index[:10])

[0 1 2 3 4 5 6 7 8 9]
[ 1  4  8 10 11 13 14 15 16 17]
[ 0  2  3  5  6  7  9 10 12 15]


In [ ]:
for train_index, test_index in kf.split(final_comb_arr):
  X_train, X_test = final_comb_arr[train_index], final_comb_arr[test_index]
  y_train, y_test = Y_train[train_index], Y_train[test_index]
  
  #cv = CountVectorizer()
  #cv_train = cv.fit_transform(X_train).toarray()
  #cv_test = cv.transform(X_test).toarray()
  train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

  test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

  BATCH_SIZE = 32
  SHUFFLE_BUFFER_SIZE = 100

  train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
  test_dataset = test_dataset.batch(BATCH_SIZE)
  get_results(get_model(X_train.shape[1]),50,train_dataset,test_dataset,y_test)

Building model...
Epoch 1/50
30/30 [==============================] - 0s 3ms/step - loss: 2.5076 - accuracy: 0.2508
Epoch 2/50
30/30 [==============================] - 0s 3ms/step - loss: 2.0880 - accuracy: 0.3908
Epoch 3/50
30/30 [==============================] - 0s 3ms/step - loss: 1.9354 - accuracy: 0.3929
Epoch 4/50
30/30 [==============================] - 0s 3ms/step - loss: 1.7712 - accuracy: 0.4692
Epoch 5/50
30/30 [==============================] - 0s 3ms/step - loss: 1.6542 - accuracy: 0.4953
Epoch 6/50
30/30 [==============================] - 0s 3ms/step - loss: 1.5941 - accuracy: 0.5047
Epoch 7/50
30/30 [==============================] - 0s 3ms/step - loss: 1.4856 - accuracy: 0.5413
Epoch 8/50
30/30 [==============================] - 0s 4ms/step - loss: 1.4242 - accuracy: 0.5705
Epoch 9/50
30/30 [==============================] - 0s 3ms/step - loss: 1.3483 - accuracy: 0.5925
Epoch 10/50
30/30 [==============================] - 0s 3ms/step - loss: 1.3270 - accuracy: 0.5893
E

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


30/30 [==============================] - 0s 3ms/step - loss: 2.4240 - accuracy: 0.2665
Epoch 2/50
30/30 [==============================] - 0s 3ms/step - loss: 1.9681 - accuracy: 0.4190
Epoch 3/50
30/30 [==============================] - 0s 3ms/step - loss: 1.7563 - accuracy: 0.4911
Epoch 4/50
30/30 [==============================] - 0s 3ms/step - loss: 1.6354 - accuracy: 0.5026
Epoch 5/50
30/30 [==============================] - 0s 3ms/step - loss: 1.5226 - accuracy: 0.5413
Epoch 6/50
30/30 [==============================] - 0s 3ms/step - loss: 1.4273 - accuracy: 0.5538
Epoch 7/50
30/30 [==============================] - 0s 3ms/step - loss: 1.3810 - accuracy: 0.5873
Epoch 8/50
30/30 [==============================] - 0s 3ms/step - loss: 1.3067 - accuracy: 0.5977
Epoch 9/50
30/30 [==============================] - 0s 3ms/step - loss: 1.2426 - accuracy: 0.6113
Epoch 10/50
30/30 [==============================] - 0s 3ms/step - loss: 1.2165 - accuracy: 0.6134
Epoch 11/50
30/30 [===========

In [ ]:
#f1 macro 45 45 45
#accuracy 54 51 58

In [ ]:
print((45+45+45)/3)
print((54+51+58)/3)

45.0
54.333333333333336


# Training with aligned mt5 small data

In [ ]:
chichewa_embd_align = np.load('../Data/' + 'embeddings_chichewa_align.npy')
english_embd_align = np.load('../Data/' + 'embeddings_english_align.npy')

In [ ]:
english_embd_align.shape

(1436, 256)

In [ ]:
final_comb_arr_align = np.hstack((chichewa_embd_align,english_embd_align))

In [ ]:
final_comb_arr.shape

(1436, 512)

In [ ]:
for train_index, test_index in kf.split(final_comb_arr_align):
  X_train, X_test = final_comb_arr_align[train_index], final_comb_arr_align[test_index]
  y_train, y_test = Y_train[train_index], Y_train[test_index]
  
  #cv = CountVectorizer()
  #cv_train = cv.fit_transform(X_train).toarray()
  #cv_test = cv.transform(X_test).toarray()
  train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

  test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

  BATCH_SIZE = 32
  SHUFFLE_BUFFER_SIZE = 100

  train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
  test_dataset = test_dataset.batch(BATCH_SIZE)
  get_results(get_model(X_train.shape[1]),50,train_dataset,test_dataset,y_test)

Building model...
Epoch 1/50
30/30 [==============================] - 0s 3ms/step - loss: 2.2644 - accuracy: 0.3386
Epoch 2/50
30/30 [==============================] - 0s 3ms/step - loss: 1.5875 - accuracy: 0.5287
Epoch 3/50
30/30 [==============================] - 0s 3ms/step - loss: 1.3684 - accuracy: 0.5883
Epoch 4/50
30/30 [==============================] - 0s 3ms/step - loss: 1.2486 - accuracy: 0.6196
Epoch 5/50
30/30 [==============================] - 0s 3ms/step - loss: 1.1579 - accuracy: 0.6479
Epoch 6/50
30/30 [==============================] - 0s 3ms/step - loss: 1.0896 - accuracy: 0.6625
Epoch 7/50
30/30 [==============================] - 0s 3ms/step - loss: 1.0120 - accuracy: 0.6876
Epoch 8/50
30/30 [==============================] - 0s 3ms/step - loss: 0.9568 - accuracy: 0.7200
Epoch 9/50
30/30 [==============================] - 0s 3ms/step - loss: 0.9059 - accuracy: 0.7346
Epoch 10/50
30/30 [==============================] - 0s 3ms/step - loss: 0.8461 - accuracy: 0.7377
E

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


30/30 [==============================] - 0s 3ms/step - loss: 2.1732 - accuracy: 0.3845
Epoch 2/50
30/30 [==============================] - 0s 3ms/step - loss: 1.4847 - accuracy: 0.5611
Epoch 3/50
30/30 [==============================] - 0s 3ms/step - loss: 1.2957 - accuracy: 0.6082
Epoch 4/50
30/30 [==============================] - 0s 3ms/step - loss: 1.1814 - accuracy: 0.6322
Epoch 5/50
30/30 [==============================] - 0s 3ms/step - loss: 1.1164 - accuracy: 0.6437
Epoch 6/50
30/30 [==============================] - 0s 3ms/step - loss: 1.0418 - accuracy: 0.6761
Epoch 7/50
30/30 [==============================] - 0s 3ms/step - loss: 0.9962 - accuracy: 0.6876
Epoch 8/50
30/30 [==============================] - 0s 3ms/step - loss: 0.9284 - accuracy: 0.7074
Epoch 9/50
30/30 [==============================] - 0s 3ms/step - loss: 0.8702 - accuracy: 0.7231
Epoch 10/50
30/30 [==============================] - 0s 3ms/step - loss: 0.8289 - accuracy: 0.7356
Epoch 11/50
30/30 [===========

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


30/30 [==============================] - 0s 3ms/step - loss: 2.1851 - accuracy: 0.3622
Epoch 2/50
30/30 [==============================] - 0s 3ms/step - loss: 1.4959 - accuracy: 0.5355
Epoch 3/50
30/30 [==============================] - 0s 3ms/step - loss: 1.2942 - accuracy: 0.5835
Epoch 4/50
30/30 [==============================] - 0s 3ms/step - loss: 1.1889 - accuracy: 0.6096
Epoch 5/50
30/30 [==============================] - 0s 3ms/step - loss: 1.1083 - accuracy: 0.6493
Epoch 6/50
30/30 [==============================] - 0s 3ms/step - loss: 1.0517 - accuracy: 0.6618
Epoch 7/50
30/30 [==============================] - 0s 3ms/step - loss: 0.9904 - accuracy: 0.6743
Epoch 8/50
30/30 [==============================] - 0s 3ms/step - loss: 0.9295 - accuracy: 0.6994
Epoch 9/50
30/30 [==============================] - 0s 3ms/step - loss: 0.8969 - accuracy: 0.7129
Epoch 10/50
30/30 [==============================] - 0s 3ms/step - loss: 0.8286 - accuracy: 0.7255
Epoch 11/50
30/30 [===========

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#f1 macro 44 40 44
#accuracy 57 54 59

In [ ]:
print((44+40+44)/3)
print((57+54+59)/3)

42.666666666666664
56.666666666666664


# Training with aligned mt5 small chichewa and webscraped data

In [ ]:
chichewa_embd_align = np.load('../Data/' + 'embeddings_chichewa_align.npy')
chichewa_embd_align_extra = np.load('../Data/' + 'web_scraped_data.ny.npy')
align_extra_labels = pd.read_csv('../Data/' + 'web_scraped_data.ny.csv')
#english_embd_align_extra = np.load('../Data/' + 'web_scraped_data_split_3.ny.npy')

In [ ]:
Y_train_align = np.copy(Y_train)

In [ ]:
le = LabelEncoder()
align_extra_labels.Label = le.fit_transform(align_extra_labels.Label)

In [ ]:
temp = np_utils.to_categorical(align_extra_labels['Label'], 20)

In [ ]:
print(Y_train_align.shape)
print(temp.shape)

(1436, 20)
(354, 20)


In [ ]:
Y_train_align = np.vstack((Y_train_align,temp))
print(Y_train_align.shape)

(1790, 20)


In [ ]:
print(chichewa_embd_align.shape)
print(chichewa_embd_align_extra.shape)
#print(english_embd_align_extra.shape)

(1436, 256)
(354, 256)


In [ ]:
final_comb_arr_align_2 = np.vstack((chichewa_embd_align,chichewa_embd_align_extra))

In [ ]:
final_comb_arr_align_2.shape

(1790, 256)

In [ ]:
for train_index, test_index in kf.split(final_comb_arr_align_2):
  X_train, X_test = final_comb_arr_align_2[train_index], final_comb_arr_align_2[test_index]
  y_train, y_test = Y_train_align[train_index], Y_train_align[test_index]
  

  train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

  test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

  BATCH_SIZE = 32
  SHUFFLE_BUFFER_SIZE = 100

  train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
  test_dataset = test_dataset.batch(BATCH_SIZE)
  get_results(get_model(X_train.shape[1]),50,train_dataset,test_dataset,y_test)

Building model...
Epoch 1/50
38/38 [==============================] - 0s 3ms/step - loss: 2.6360 - accuracy: 0.2137
Epoch 2/50
38/38 [==============================] - 0s 3ms/step - loss: 1.9453 - accuracy: 0.4392
Epoch 3/50
38/38 [==============================] - 0s 3ms/step - loss: 1.5883 - accuracy: 0.5474
Epoch 4/50
38/38 [==============================] - 0s 3ms/step - loss: 1.4032 - accuracy: 0.5733
Epoch 5/50
38/38 [==============================] - 0s 3ms/step - loss: 1.2794 - accuracy: 0.6119
Epoch 6/50
38/38 [==============================] - 0s 3ms/step - loss: 1.1984 - accuracy: 0.6320
Epoch 7/50
38/38 [==============================] - 0s 3ms/step - loss: 1.1314 - accuracy: 0.6513
Epoch 8/50
38/38 [==============================] - 0s 3ms/step - loss: 1.0742 - accuracy: 0.6723
Epoch 9/50
38/38 [==============================] - 0s 3ms/step - loss: 1.0254 - accuracy: 0.6857
Epoch 10/50
38/38 [==============================] - 0s 3ms/step - loss: 0.9767 - accuracy: 0.6940
E

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


38/38 [==============================] - 0s 3ms/step - loss: 2.6088 - accuracy: 0.2230
Epoch 2/50
38/38 [==============================] - 0s 3ms/step - loss: 1.9261 - accuracy: 0.4535
Epoch 3/50
38/38 [==============================] - 0s 3ms/step - loss: 1.5611 - accuracy: 0.5507
Epoch 4/50
38/38 [==============================] - 0s 3ms/step - loss: 1.3768 - accuracy: 0.5826
Epoch 5/50
38/38 [==============================] - 0s 3ms/step - loss: 1.2684 - accuracy: 0.6069
Epoch 6/50
38/38 [==============================] - 0s 3ms/step - loss: 1.1727 - accuracy: 0.6463
Epoch 7/50
38/38 [==============================] - 0s 2ms/step - loss: 1.1183 - accuracy: 0.6622
Epoch 8/50
38/38 [==============================] - 0s 3ms/step - loss: 1.0541 - accuracy: 0.6731
Epoch 9/50
38/38 [==============================] - 0s 3ms/step - loss: 1.0185 - accuracy: 0.6781
Epoch 10/50
38/38 [==============================] - 0s 3ms/step - loss: 0.9818 - accuracy: 0.6882
Epoch 11/50
38/38 [===========

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#f1 macro 52 52 50
#accuracy 63 59 61

In [ ]:
print((52+52+50)/3)
print((63+59+61)/3)

51.333333333333336
61.0


# Training with mt5 small aligned chichewa data and 3 top sentence web scraped data

In [ ]:
chichewa_embd_align = np.load('../Data/' + 'embeddings_chichewa_align.npy')
chichewa_embd_align_extra_split = np.load('../Data/' + 'web_scraped_data_split_3.ny.npy')
align_extra_labels_split = pd.read_csv('../Data/' + 'web_scraped_data_split_3.ny.csv')

In [ ]:
print(Y_train.shape)

(1436, 20)


In [ ]:
Y_train_align = np.copy(Y_train)

In [ ]:
le = LabelEncoder()
align_extra_labels_split.Label = le.fit_transform(align_extra_labels_split.Label)

In [ ]:
temp = np_utils.to_categorical(align_extra_labels_split['Label'], 20)

In [ ]:
print(Y_train_align.shape)
print(temp.shape)

(1436, 20)
(2708, 20)


In [ ]:
Y_train_align = np.vstack((Y_train_align,temp))
print(Y_train_align.shape)

(4144, 20)


In [ ]:
print(chichewa_embd_align.shape)
print(chichewa_embd_align_extra_split.shape)
#print(english_embd_align_extra.shape)

(1436, 256)
(2708, 256)


In [ ]:
final_comb_arr_align_2 = np.vstack((chichewa_embd_align,chichewa_embd_align_extra_split))

In [ ]:
final_comb_arr_align_2.shape

(4144, 256)

In [ ]:
for train_index, test_index in kf.split(final_comb_arr_align_2):
  X_train, X_test = final_comb_arr_align_2[train_index], final_comb_arr_align_2[test_index]
  y_train, y_test = Y_train_align[train_index], Y_train_align[test_index]
  

  train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

  test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

  BATCH_SIZE = 32
  SHUFFLE_BUFFER_SIZE = 100

  train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
  test_dataset = test_dataset.batch(BATCH_SIZE)
  get_results(get_model(X_train.shape[1]),50,train_dataset,test_dataset,y_test)

Building model...
Epoch 1/50
87/87 [==============================] - 1s 3ms/step - loss: 2.5803 - accuracy: 0.2918
Epoch 2/50
87/87 [==============================] - 0s 3ms/step - loss: 1.6717 - accuracy: 0.4562
Epoch 3/50
87/87 [==============================] - 0s 3ms/step - loss: 1.4088 - accuracy: 0.5543
Epoch 4/50
87/87 [==============================] - 0s 3ms/step - loss: 1.1518 - accuracy: 0.6224
Epoch 5/50
87/87 [==============================] - 0s 3ms/step - loss: 0.9973 - accuracy: 0.6767
Epoch 6/50
87/87 [==============================] - 0s 3ms/step - loss: 0.8746 - accuracy: 0.7075
Epoch 7/50
87/87 [==============================] - 0s 3ms/step - loss: 0.7891 - accuracy: 0.7393
Epoch 8/50
87/87 [==============================] - 0s 3ms/step - loss: 0.7277 - accuracy: 0.7574
Epoch 9/50
87/87 [==============================] - 0s 3ms/step - loss: 0.6784 - accuracy: 0.7777
Epoch 10/50
87/87 [==============================] - 0s 3ms/step - loss: 0.6434 - accuracy: 0.7867
E

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


87/87 [==============================] - 1s 5ms/step - loss: 2.5395 - accuracy: 0.2819
Epoch 2/50
87/87 [==============================] - 0s 5ms/step - loss: 1.6743 - accuracy: 0.4499
Epoch 3/50
87/87 [==============================] - 0s 5ms/step - loss: 1.3919 - accuracy: 0.5574
Epoch 4/50
87/87 [==============================] - 0s 5ms/step - loss: 1.1486 - accuracy: 0.6298
Epoch 5/50
87/87 [==============================] - 0s 5ms/step - loss: 1.0036 - accuracy: 0.6710
Epoch 6/50
87/87 [==============================] - 0s 5ms/step - loss: 0.8989 - accuracy: 0.7105
Epoch 7/50
87/87 [==============================] - 0s 4ms/step - loss: 0.8086 - accuracy: 0.7365
Epoch 8/50
87/87 [==============================] - 1s 6ms/step - loss: 0.7498 - accuracy: 0.7561
Epoch 9/50
87/87 [==============================] - 0s 4ms/step - loss: 0.7026 - accuracy: 0.7695
Epoch 10/50
87/87 [==============================] - 0s 4ms/step - loss: 0.6564 - accuracy: 0.7847
Epoch 11/50
87/87 [===========

In [ ]:
#f1 macro 56 56 51
#accuracy 77 78 77

In [ ]:
print((56+56+51)/3)
print((77+78+77)/3)

54.333333333333336
77.33333333333333


In [ ]:
Y_train.shape

(1436, 20)

# Training with mt5 small aligned chichewa data and contextual word subsitution data

In [ ]:
#rus = RandomUnderSampler(random_state=42)

In [ ]:
chichewa_embd_align = np.load('../Data/' + 'embeddings_chichewa_align.npy')
chichewa_embd_align_extra_split = np.load('../Data/' + 'augmented_data_split_3.ny.npy')
align_extra_labels_split = pd.read_csv('../Data/' + 'augmented_data_split_3.ny.csv')

print(Y_train.shape)
print('\n')

Y_train_align = np.copy(Y_train)
le = LabelEncoder()
align_extra_labels_split.Label = le.fit_transform(align_extra_labels_split.Label)
temp = np_utils.to_categorical(align_extra_labels_split['Label'], 20)
print(Y_train_align.shape)
print(temp.shape)
Y_train_align = np.vstack((Y_train_align,temp))
print(Y_train_align.shape)
print('\n')

print(chichewa_embd_align.shape)
print(chichewa_embd_align_extra_split.shape)
final_comb_arr_align_2 = np.vstack((chichewa_embd_align,chichewa_embd_align_extra_split))
final_comb_arr_align_2.shape

#X_res, y_res = rus.fit_resample(final_comb_arr_align_2, Y_train_align)

(1436, 20)


(1436, 20)
(28919, 20)
(30355, 20)


(1436, 256)
(28919, 256)


(30355, 256)

In [ ]:
for train_index, test_index in kf.split(final_comb_arr_align_2):
  X_train, X_test = final_comb_arr_align_2[train_index], final_comb_arr_align_2[test_index]
  y_train, y_test = Y_train_align[train_index], Y_train_align[test_index]
  

  train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

  test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

  BATCH_SIZE = 32
  SHUFFLE_BUFFER_SIZE = 100

  train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
  test_dataset = test_dataset.batch(BATCH_SIZE)
  get_results(get_model(X_train.shape[1]),50,train_dataset,test_dataset,y_test)

Building model...
Epoch 1/50
633/633 [==============================] - 2s 3ms/step - loss: 1.9499 - accuracy: 0.5159
Epoch 2/50
633/633 [==============================] - 2s 3ms/step - loss: 1.2422 - accuracy: 0.6586
Epoch 3/50
633/633 [==============================] - 2s 3ms/step - loss: 1.1110 - accuracy: 0.7028
Epoch 4/50
633/633 [==============================] - 2s 3ms/step - loss: 1.0381 - accuracy: 0.7167
Epoch 5/50
633/633 [==============================] - 2s 3ms/step - loss: 0.9803 - accuracy: 0.7247
Epoch 6/50
633/633 [==============================] - 2s 3ms/step - loss: 0.9634 - accuracy: 0.7258
Epoch 7/50
633/633 [==============================] - 2s 3ms/step - loss: 0.9065 - accuracy: 0.7305
Epoch 8/50
633/633 [==============================] - 2s 3ms/step - loss: 0.8911 - accuracy: 0.7349
Epoch 9/50
633/633 [==============================] - 2s 3ms/step - loss: 0.8179 - accuracy: 0.7499
Epoch 10/50
633/633 [==============================] - 2s 3ms/step - loss: 0.7929 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#f1 macro 50 49 51
#accuracy 60 58 60

In [ ]:
print((50+49+51)/3)
print((60+58+60)/3)

50.0
59.333333333333336


# Training with mixup unaligned embeddings

In [ ]:
chichewa_embd_align = np.load('../Data/mixup_embeddings/MT5_mixup_embeddings.npy')
align_extra_labels_split = pd.read_csv('../Data/mixup_embeddings/MT5_mixup_labels.csv')
print(Y_train.shape)
print('\n')

#Y_train_align = np.copy(Y_train)
le = LabelEncoder()
align_extra_labels_split['0'] = le.fit_transform(align_extra_labels_split['0'])
Y_train_align = np.copy(np_utils.to_categorical(align_extra_labels_split['0'], 20))
print(Y_train_align.shape)
print('\n')

print(chichewa_embd_align.shape)
final_comb_arr_align_2 = np.copy(chichewa_embd_align)
#final_comb_arr_align_2.shape

#X_res, y_res = rus.fit_resample(final_comb_arr_align_2, Y_train_align)

(1436, 20)


(49427, 20)


(49427, 256)


In [ ]:
for train_index, test_index in kf.split(final_comb_arr_align_2):
  X_train, X_test = final_comb_arr_align_2[train_index], final_comb_arr_align_2[test_index]
  y_train, y_test = Y_train_align[train_index], Y_train_align[test_index]
  #print(X_train.shape)
  #print(y_train.shape)
  #break

  train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

  test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

  BATCH_SIZE = 32
  SHUFFLE_BUFFER_SIZE = 100

  train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
  test_dataset = test_dataset.batch(BATCH_SIZE)
  get_results(get_model(X_train.shape[1]),50,train_dataset,test_dataset,y_test)

Building model...
Epoch 1/50
1030/1030 [==============================] - 3s 3ms/step - loss: 1.5496 - accuracy: 0.6787
Epoch 2/50
1030/1030 [==============================] - 3s 3ms/step - loss: 0.7719 - accuracy: 0.8089
Epoch 3/50
1030/1030 [==============================] - 3s 3ms/step - loss: 0.5753 - accuracy: 0.8537
Epoch 4/50
1030/1030 [==============================] - 3s 3ms/step - loss: 0.4674 - accuracy: 0.8832
Epoch 5/50
1030/1030 [==============================] - 3s 3ms/step - loss: 0.4111 - accuracy: 0.8998
Epoch 6/50
1030/1030 [==============================] - 3s 3ms/step - loss: 0.3689 - accuracy: 0.9070
Epoch 7/50
1030/1030 [==============================] - 3s 3ms/step - loss: 0.3256 - accuracy: 0.9210
Epoch 8/50
1030/1030 [==============================] - 3s 3ms/step - loss: 0.2982 - accuracy: 0.9275
Epoch 9/50
1030/1030 [==============================] - 3s 3ms/step - loss: 0.2634 - accuracy: 0.9360
Epoch 10/50
1030/1030 [==============================] - 3s 3ms/

In [ ]:
#f1 score: 79 81 80
#accuracy: 80 81 80

In [ ]:
print((79+81+80)/3)
print((80+81+80)/3)

80.0
80.33333333333333


In [ ]:
%ls ../Data/mixup_embeddings/

data/  final_embeddings.npy  final_labels.npy  labels/
